# Libraries

In [1]:
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
tf.get_logger().setLevel(3)

2023-05-01 19:45:48.081078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 19:45:48.244392: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-01 19:45:48.756215: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-01 19:45:48.756319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Building the Neural Network

In [2]:
def create_cnn(width=154, height=154, depth=3, filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(10)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model  

# Preprocessing and converting the data to train and test

In [3]:
data_path = 'dataset/'

X_train = []
X_test = []
Y_train = []
Y_test = []

for dir in os.listdir(data_path):
    for classes in os.listdir(data_path+'/'+ dir):
        for file in os.listdir(data_path+'/'+dir+'/'+classes):
            fmt = file.split('.')

            if fmt[-1] == "jpg":
                img_path = data_path+'/'+dir+'/'+classes+'/'+file
                img = np.array(Image.open(img_path))
                if dir == 'train':
                    X_train.append(img)
                    Y_train.append(int(classes))
                elif dir == 'test':
                    X_test.append(img)
                    Y_test.append(int(classes))

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

Y = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y[Y_train[i]//10,i] = 1

# Training the model

In [4]:
model = create_cnn(154, 154, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=X_train, y=Y.T, epochs=1000, batch_size=50, verbose=0)

2023-05-01 19:45:50.455084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-01 19:45:50.504312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-01 19:45:50.504379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-01 19:45:50.504918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

# Testing the trained model

In [5]:
preds = model.predict(X_test)
y_pred = np.zeros(len(preds))

for i in range(0,len(preds)):
    y_pred[i] = 10 * np.argmax(preds[i,:])

# print("True values: ", Y_test)
# print("Predicted values:", y_pred)
print("Confusion Matrix :")
print(confusion_matrix(Y_test, y_pred))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_test, y_pred)))

5/5 [==============================] - 1s 36ms/step
Confusion Matrix :
[[ 9  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  1  0  0  0  0  0  0  0  0]
 [ 0  1  3  9  0  0  0  0  0  0  0]
 [ 0  0  0  8  1  1  0  0  0  0  0]
 [ 0  0  0  6  7  2  0  0  0  0  0]
 [ 2  0  0  0  4  5  2  0  0  0  0]
 [ 1  0  0  0  0 10  6  1  0  0  0]
 [ 2  1  0  0  0  8  9  3  0  0  0]
 [ 0  0  0  0  0  4  2  5  3  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  4  1  1  3 10  0]]

Accuracy: 0.35

